In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("prenoms.csv")

In [ ]:
df = pd.read_csv('prenoms.csv')
df['name'] = df['name'].str.lower()
df = df.groupby('name', as_index=False).agg({'count': 'sum'})
df = df.sample(frac=1.0)
df = df[df['count'] >= 50]
df = df[:100]

In [ ]:
df

In [ ]:
names = df['name'].tolist()
names

In [2]:
chars = sorted(set(''.join(names)))

NameError: name 'names' is not defined

In [ ]:
chars = sorted(set(''.join(names)))
char_to_index = {s:i+1 for i,s in enumerate(chars)}
char_to_index['.'] = 0
num_chars = len(char_to_index)
print(f"Nombre de caractères: {num_chars}")
index_to_char = {i:c for c,i in char_to_index.items()}

In [ ]:
import torch

In [ ]:
xs, ys = [],[]

for name in names[:1]:
  chs= ['.']+ list(name)+['.']
  for ch1, ch2 in zip(chs, chs[1:]): # zip (prenom, prenom décalé de 1)
    print(ch1, ch2)
    xs.append(char_to_index[ch1])
    ys.append(char_to_index[ch2])

In [ ]:
xs = torch.tensor(xs)
ys = torch.tensor(ys)

num = xs.nelement()

print(f"Nombre d'exemple : {num}")

In [ ]:
import torch.nn.functional as F

In [ ]:
# exemple d'utilisation de conversion one_hot pour la valeur 1 sur 5 bits
F.one_hot(torch.tensor(1), num_classes=5)

In [ ]:
# xs encodé en one_hot
xenc = F.one_hot(xs, num_classes=num_chars) # encodage sur le nombre possible de caractere

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(xenc)

In [ ]:
# creation du reseau de neuronne
W = torch.randn((num_chars, num_chars), requires_grad=True) # requires_grad peut etre entrainé

In [ ]:
# calcul manuel de l'entrainement : multiplication de matrice
%%time
output = torch.zeros((xenc.shape[0], W.shape[1]))
for i in range(xenc.shape[0]):
    for j in range(W.shape[1]):
        for k in range(W.shape[0]):
            output[i, j] += xenc[i, k] * W[k, j]

output

In [ ]:
xenc = xenc.float()
xenc @ W  # multiplication de matrice avec pytorch

In [ ]:
# pour avoir que des valeurs positives exp() (exponentiel), on divise par la sum pour normalisé entre 0 et 1
# voir function softmax
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
probs


In [ ]:
nlls = torch.zeros(num)
for i in range(num):
  x = xs[i].item() # caractère d'entrée
  y = ys[i].item() # caractère label
  print('--------')
  print(f'bigramme {i+1}: {index_to_char[x]}{index_to_char[y]} (indexes {x}, {y})')
  print('entrée du réseau:', x)
  print('probabilités en sortie du réseau:', probs[i])
  print('label (caractère de sortie "correct"):', y)
  p = probs[i, y]
  print('probabilité assigné à ce vrai caractère par le réseau:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood (loss) =', nlls.mean().item())
print(f'Perplexité: {torch.exp(nlls.mean()):.2f}')

In [ ]:
notre reseau de neuronne est mauvais

In [ ]:
xs

In [ ]:
ys

In [ ]:
probs[0,12]

In [ ]:
probs[torch.arange(num), ys] # donne un tableau des probabilités vrais qu'il faudrait augmentée

In [ ]:
loss = -probs[torch.arange(num), ys].log.mean()
loss.item() # donne la valeur d'un tenseur pytorch


In [ ]:
# mise a jour des poid via pytorch
loss.backward()  # entrainement de W

In [ ]:
W.grad #comment devrait etre changer W (difference a appliqué a W pour l'optimiser)

In [ ]:
W.data += -0.1* W.grad # applique le changement mais pas trop vite donc on minimise le changement par 0.1

In [ ]:

from tqdm import tqdm
# entrainement en boucle

# on entraine en boucle
for k in range(500):
  # forward pass
  xenc = F.one_hot(xs, num_classes=num_chars).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean()
  print(loss.item())

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update
  W.data += -50 * W.grad

In [ ]:
# generation d'un prenom a partir de notre model

for i in range(5):
  out = []
  ix = 0
  while True:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=num_chars).float()
    logits = xenc @ W
    counts = logits.exp()
    p = counts / counts.sum(1, keepdims=True)

    ix = torch.multinomial(p, num_samples=1, replacement=True).item()
    out.append(index_to_char[ix])
    if ix == 0:
      break
  print(''.join(out))